In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting

#%reload_ext autoreload
#%autoreload 2
%matplotlib inline

import fastai
from fastai.train import Learner
from fastai.train import DataBunch
from fastai.callbacks import *
from fastai.basic_data import DatasetType
import fastprogress
from fastprogress import force_console_behavior
import numpy as np
from pprint import pprint
import pandas as pd
import time


import gc
from tqdm._tqdm_notebook import tqdm_notebook as tqdm
from keras.preprocessing import text, sequence
import torch
from torch import nn
from torch.utils import data
from torch.nn import functional as F
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold, StratifiedKFold
notebook_start_time = time.time()
tqdm.pandas() # for progress_apply

from jig_utils import (JigsawEvaluator,seed_everything, get_coefs, load_embeddings,
                       make_weights_dir, sigmoid, SpatialDropout, save_model_stats,
                       SequenceBucketCollator, threshold_search)

seed_everything(123)

Using TensorFlow backend.


** To replace the pretrained embedding files with their pickle corresponds. Loading a pickled version (most other public kernels)**

In [2]:
MODEL_NOTES = ""
MODEL_DIR = "../models/v5_3_2_1/"
CRAWL_EMBEDDING_PATH = '../input/crawl-300d-2M.pkl'
GLOVE_EMBEDDING_PATH = '../input/glove.840B.300d.pkl'

max_word_len = 300 # the max num words for each sentence

NUM_MODELS = 2
LSTM_UNITS = 128
DENSE_HIDDEN_UNITS = 4 * LSTM_UNITS
MAX_LEN = 220

batch_size = 512
NUM_SPLITS = 2

SMALL_DATA = True

* To adjust the load_embeddings function (Pickled dict).

* To try a "lower/upper case version of a" word if an embedding is not found, which sometimes gives us an embedding

In [3]:
# TODO: modify this with quora build matrix
def build_matrix(word_index, path):
    embedding_index = load_embeddings(path)
    embedding_matrix = np.zeros((max_features + 1, 300))
    unknown_words = []
    
    for word, i in word_index.items():
        if i <= max_features:
            try:
                embedding_matrix[i] = embedding_index[word]
            except KeyError:
                try:
                    embedding_matrix[i] = embedding_index[word.lower()]
                except KeyError:
                    try:
                        embedding_matrix[i] = embedding_index[word.title()]
                    except KeyError:
                        unknown_words.append(word)
    return embedding_matrix, unknown_words

In [4]:
def train_model(learn,valid,output_dim,lr=0.001,
                batch_size=512, n_epochs=5,
                enable_checkpoint_ensemble=False):
    
    all_valid_preds = []
    checkpoint_weights = [1,2,4,8,6]
    valid_loader = torch.utils.data.DataLoader(valid, batch_size=batch_size, shuffle=False)
    n = len(learn.data.train_dl)
    phases = [(TrainingPhase(n).schedule_hp('lr', lr * (0.6**(i)))) for i in range(n_epochs)]
    sched = GeneralScheduler(learn, phases)
    learn.callbacks.append(sched)
    for epoch in range(n_epochs):
        learn.fit(1)  
        valid_preds = np.zeros((len(valid), output_dim))
        
        for i, x_batch in enumerate(valid_loader):
            X = x_batch[0].cuda()
            y_pred = sigmoid(learn.model(X).detach().cpu().numpy())
            valid_preds[i * batch_size:(i+1) * batch_size, :] = y_pred

        all_valid_preds.append(valid_preds)


    if enable_checkpoint_ensemble: 
        valid_preds = np.average(all_valid_preds, weights=checkpoint_weights, axis=0)
    else:
        valid_preds = all_valid_preds[-1]
        
    del valid_loader
    gc.collect()
        
    return valid_preds

In [5]:
class NeuralNet(nn.Module):
    def __init__(self, embedding_matrix, num_aux_targets):
        super(NeuralNet, self).__init__()
        embed_size = embedding_matrix.shape[1]
        
        self.embedding = nn.Embedding(max_features, embed_size)
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = False
        self.embedding_dropout = SpatialDropout(0.3)
        
        self.lstm1 = nn.LSTM(embed_size, LSTM_UNITS, bidirectional=True, batch_first=True)
        self.lstm2 = nn.LSTM(LSTM_UNITS * 2, LSTM_UNITS, bidirectional=True, batch_first=True)
    
        self.linear1 = nn.Linear(DENSE_HIDDEN_UNITS, DENSE_HIDDEN_UNITS)
        self.linear2 = nn.Linear(DENSE_HIDDEN_UNITS, DENSE_HIDDEN_UNITS)
        
        self.linear_out = nn.Linear(DENSE_HIDDEN_UNITS, 1)
        self.linear_aux_out = nn.Linear(DENSE_HIDDEN_UNITS, num_aux_targets)
        
    def forward(self, x, lengths=None):
        #TODO: doesn't x.long() cast to int64??? will this affect apex?
        h_embedding = self.embedding(x.long())
        h_embedding = self.embedding_dropout(h_embedding)
        
        h_lstm1, _ = self.lstm1(h_embedding)
        h_lstm2, _ = self.lstm2(h_lstm1)
        
        # global average pooling
        avg_pool = torch.mean(h_lstm2, 1)
        # global max pooling
        max_pool, _ = torch.max(h_lstm2, 1)
        
        h_conc = torch.cat((max_pool, avg_pool), 1)
        h_conc_linear1  = F.relu(self.linear1(h_conc))
        h_conc_linear2  = F.relu(self.linear2(h_conc))
        
        hidden = h_conc + h_conc_linear1 + h_conc_linear2
        
        result = self.linear_out(hidden)
        aux_result = self.linear_aux_out(hidden)
        out = torch.cat([result, aux_result], 1)
        
        return out

## Preprocessing

In [6]:
train = pd.read_hdf('../input/train.h5')#[:1000]
test = pd.read_hdf('../input/test.h5')#[:1000]
if(SMALL_DATA):
    train = train[:1000]
    test = test[:1000]

In [7]:
symbols_to_isolate = '.,?!-;*"…:—()%#$&_/@＼・ω+=”“[]^–>\\°<~•≠™ˈʊɒ∞§{}·τα❤☺ɡ|¢→̶`❥━┣┫┗Ｏ►★©―ɪ✔®\x96\x92●£♥➤´¹☕≈÷♡◐║▬′ɔː€۩۞†μ✒➥═☆ˌ◄½ʻπδηλσερνʃ✬ＳＵＰＥＲＩＴ☻±♍µº¾✓◾؟．⬅℅»Вав❣⋅¿¬♫ＣＭβ█▓▒░⇒⭐›¡₂₃❧▰▔◞▀▂▃▄▅▆▇↙γ̄″☹➡«φ⅓„✋：¥̲̅́∙‛◇✏▷❓❗¶˚˙）сиʿ✨。ɑ\x80◕！％¯−ﬂﬁ₁²ʌ¼⁴⁄₄⌠♭✘╪▶☭✭♪☔☠♂☃☎✈✌✰❆☙○‣⚓年∎ℒ▪▙☏⅛ｃａｓǀ℮¸ｗ‚∼‖ℳ❄←☼⋆ʒ⊂、⅔¨͡๏⚾⚽Φ×θ￦？（℃⏩☮⚠月✊❌⭕▸■⇌☐☑⚡☄ǫ╭∩╮，例＞ʕɐ̣Δ₀✞┈╱╲▏▕┃╰▊▋╯┳┊≥☒↑☝ɹ✅☛♩☞ＡＪＢ◔◡↓♀⬆̱ℏ\x91⠀ˤ╚↺⇤∏✾◦♬³の｜／∵∴√Ω¤☜▲↳▫‿⬇✧ｏｖｍ－２０８＇‰≤∕ˆ⚜☁'
symbols_to_delete = '\n🍕\r🐵😑\xa0\ue014\t\uf818\uf04a\xad😢🐶️\uf0e0😜😎👊\u200b\u200e😁عدويهصقأناخلىبمغر😍💖💵Е👎😀😂\u202a\u202c🔥😄🏻💥ᴍʏʀᴇɴᴅᴏᴀᴋʜᴜʟᴛᴄᴘʙғᴊᴡɢ😋👏שלוםבי😱‼\x81エンジ故障\u2009🚌ᴵ͞🌟😊😳😧🙀😐😕\u200f👍😮😃😘אעכח💩💯⛽🚄🏼ஜ😖ᴠ🚲‐😟😈💪🙏🎯🌹😇💔😡\x7f👌ἐὶήιὲκἀίῃἴξ🙄Ｈ😠\ufeff\u2028😉😤⛺🙂\u3000تحكسة👮💙فزط😏🍾🎉😞\u2008🏾😅😭👻😥😔😓🏽🎆🍻🍽🎶🌺🤔😪\x08‑🐰🐇🐱🙆😨🙃💕𝘊𝘦𝘳𝘢𝘵𝘰𝘤𝘺𝘴𝘪𝘧𝘮𝘣💗💚地獄谷улкнПоАН🐾🐕😆ה🔗🚽歌舞伎🙈😴🏿🤗🇺🇸мυтѕ⤵🏆🎃😩\u200a🌠🐟💫💰💎эпрд\x95🖐🙅⛲🍰🤐👆🙌\u2002💛🙁👀🙊🙉\u2004ˢᵒʳʸᴼᴷᴺʷᵗʰᵉᵘ\x13🚬🤓\ue602😵άοόςέὸתמדףנרךצט😒͝🆕👅👥👄🔄🔤👉👤👶👲🔛🎓\uf0b7\uf04c\x9f\x10成都😣⏺😌🤑🌏😯ех😲Ἰᾶὁ💞🚓🔔📚🏀👐\u202d💤🍇\ue613小土豆🏡❔⁉\u202f👠》कर्मा🇹🇼🌸蔡英文🌞🎲レクサス😛外国人关系Сб💋💀🎄💜🤢َِьыгя不是\x9c\x9d🗑\u2005💃📣👿༼つ༽😰ḷЗз▱ц￼🤣卖温哥华议会下降你失去所有的钱加拿大坏税骗子🐝ツ🎅\x85🍺آإشء🎵🌎͟ἔ油别克🤡🤥😬🤧й\u2003🚀🤴ʲшчИОРФДЯМюж😝🖑ὐύύ特殊作戦群щ💨圆明园קℐ🏈😺🌍⏏ệ🍔🐮🍁🍆🍑🌮🌯🤦\u200d𝓒𝓲𝓿𝓵안영하세요ЖљКћ🍀😫🤤ῦ我出生在了可以说普通话汉语好极🎼🕺🍸🥂🗽🎇🎊🆘🤠👩🖒🚪天一家⚲\u2006⚭⚆⬭⬯⏖新✀╌🇫🇷🇩🇪🇮🇬🇧😷🇨🇦ХШ🌐\x1f杀鸡给猴看ʁ𝗪𝗵𝗲𝗻𝘆𝗼𝘂𝗿𝗮𝗹𝗶𝘇𝗯𝘁𝗰𝘀𝘅𝗽𝘄𝗱📺ϖ\u2000үսᴦᎥһͺ\u2007հ\u2001ɩｙｅ൦ｌƽｈ𝐓𝐡𝐞𝐫𝐮𝐝𝐚𝐃𝐜𝐩𝐭𝐢𝐨𝐧Ƅᴨןᑯ໐ΤᏧ௦Іᴑ܁𝐬𝐰𝐲𝐛𝐦𝐯𝐑𝐙𝐣𝐇𝐂𝐘𝟎ԜТᗞ౦〔Ꭻ𝐳𝐔𝐱𝟔𝟓𝐅🐋ﬃ💘💓ё𝘥𝘯𝘶💐🌋🌄🌅𝙬𝙖𝙨𝙤𝙣𝙡𝙮𝙘𝙠𝙚𝙙𝙜𝙧𝙥𝙩𝙪𝙗𝙞𝙝𝙛👺🐷ℋ𝐀𝐥𝐪🚶𝙢Ἱ🤘ͦ💸ج패티Ｗ𝙇ᵻ👂👃ɜ🎫\uf0a7БУі🚢🚂ગુજરાતીῆ🏃𝓬𝓻𝓴𝓮𝓽𝓼☘﴾̯﴿₽\ue807𝑻𝒆𝒍𝒕𝒉𝒓𝒖𝒂𝒏𝒅𝒔𝒎𝒗𝒊👽😙\u200cЛ‒🎾👹⎌🏒⛸公寓养宠物吗🏄🐀🚑🤷操美𝒑𝒚𝒐𝑴🤙🐒欢迎来到阿拉斯ספ𝙫🐈𝒌𝙊𝙭𝙆𝙋𝙍𝘼𝙅ﷻ🦄巨收赢得白鬼愤怒要买额ẽ🚗🐳𝟏𝐟𝟖𝟑𝟕𝒄𝟗𝐠𝙄𝙃👇锟斤拷𝗢𝟳𝟱𝟬⦁マルハニチロ株式社⛷한국어ㄸㅓ니͜ʖ𝘿𝙔₵𝒩ℯ𝒾𝓁𝒶𝓉𝓇𝓊𝓃𝓈𝓅ℴ𝒻𝒽𝓀𝓌𝒸𝓎𝙏ζ𝙟𝘃𝗺𝟮𝟭𝟯𝟲👋🦊多伦🐽🎻🎹⛓🏹🍷🦆为和中友谊祝贺与其想象对法如直接问用自己猜本传教士没积唯认识基督徒曾经让相信耶稣复活死怪他但当们聊些政治题时候战胜因圣把全堂结婚孩恐惧且栗谓这样还♾🎸🤕🤒⛑🎁批判检讨🏝🦁🙋😶쥐스탱트뤼도석유가격인상이경제황을렵게만들지않록잘관리해야합다캐나에서대마초와화약금의품런성분갈때는반드시허된사용🔫👁凸ὰ💲🗯𝙈Ἄ𝒇𝒈𝒘𝒃𝑬𝑶𝕾𝖙𝖗𝖆𝖎𝖌𝖍𝖕𝖊𝖔𝖑𝖉𝖓𝖐𝖜𝖞𝖚𝖇𝕿𝖘𝖄𝖛𝖒𝖋𝖂𝕴𝖟𝖈𝕸👑🚿💡知彼百\uf005𝙀𝒛𝑲𝑳𝑾𝒋𝟒😦𝙒𝘾𝘽🏐𝘩𝘨ὼṑ𝑱𝑹𝑫𝑵𝑪🇰🇵👾ᓇᒧᔭᐃᐧᐦᑳᐨᓃᓂᑲᐸᑭᑎᓀᐣ🐄🎈🔨🐎🤞🐸💟🎰🌝🛳点击查版🍭𝑥𝑦𝑧ＮＧ👣\uf020っ🏉ф💭🎥Ξ🐴👨🤳🦍\x0b🍩𝑯𝒒😗𝟐🏂👳🍗🕉🐲چی𝑮𝗕𝗴🍒ꜥⲣⲏ🐑⏰鉄リ事件ї💊「」\uf203\uf09a\uf222\ue608\uf202\uf099\uf469\ue607\uf410\ue600燻製シ虚偽屁理屈Г𝑩𝑰𝒀𝑺🌤𝗳𝗜𝗙𝗦𝗧🍊ὺἈἡχῖΛ⤏🇳𝒙ψՁմեռայինրւդձ冬至ὀ𝒁🔹🤚🍎𝑷🐂💅𝘬𝘱𝘸𝘷𝘐𝘭𝘓𝘖𝘹𝘲𝘫کΒώ💢ΜΟΝΑΕ🇱♲𝝈↴💒⊘Ȼ🚴🖕🖤🥘📍👈➕🚫🎨🌑🐻𝐎𝐍𝐊𝑭🤖🎎😼🕷ｇｒｎｔｉｄｕｆｂｋ𝟰🇴🇭🇻🇲𝗞𝗭𝗘𝗤👼📉🍟🍦🌈🔭《🐊🐍\uf10aლڡ🐦\U0001f92f\U0001f92a🐡💳ἱ🙇𝗸𝗟𝗠𝗷🥜さようなら🔼'

In [8]:
from nltk.tokenize.treebank import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()


isolate_dict = {ord(c):f' {c} ' for c in symbols_to_isolate}
remove_dict = {ord(c):f'' for c in symbols_to_delete}


def handle_punctuation(x):
    x = x.translate(remove_dict)
    x = x.translate(isolate_dict)
    return x

def handle_contractions(x):
    x = tokenizer.tokenize(x)
    return x

def fix_quote(x):
    x = [x_[1:] if x_.startswith("'") else x_ for x_ in x]
    x = ' '.join(x)
    return x

def preprocess(x):
    x = handle_punctuation(x)
    x = handle_contractions(x)
    x = fix_quote(x)
    return x

In [9]:
x_train = train['comment_text'].progress_apply(lambda x:preprocess(x))
y_aux_train = train[['severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat', 'sexual_explicit']]
x_test = test['comment_text'].progress_apply(lambda x:preprocess(x))

identity_columns = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness']

# Overall
weights = np.ones((len(x_train),)) / 4

# Subgroup
weights += (train[identity_columns].fillna(0).values>=0.5).sum(axis=1).astype(bool).astype(np.int) / 4

# Background Positive, Subgroup Negative
weights += (( (train['target'].values>=0.5).astype(bool).astype(np.int) +
   (train[identity_columns].fillna(0).values<0.5).sum(axis=1).astype(bool).astype(np.int) ) > 1 ).astype(bool).astype(np.int) / 4

# Background Negative, Subgroup Positive
weights += (( (train['target'].values<0.5).astype(bool).astype(np.int) +
   (train[identity_columns].fillna(0).values>=0.5).sum(axis=1).astype(bool).astype(np.int) ) > 1 ).astype(bool).astype(np.int) / 4
loss_weight = 1.0 / weights.mean()

y_train = np.vstack([(train['target'].values>=0.5).astype(np.int),weights]).T

max_features = 410047

In [10]:
tokenizer = text.Tokenizer(num_words = max_features, filters='',lower=False)
tokenizer.fit_on_texts(list(x_train) + list(x_test))

crawl_matrix, unknown_words_crawl = build_matrix(tokenizer.word_index, CRAWL_EMBEDDING_PATH)
print('n unknown words (crawl): ', len(unknown_words_crawl))

glove_matrix, unknown_words_glove = build_matrix(tokenizer.word_index, GLOVE_EMBEDDING_PATH)
print('n unknown words (glove): ', len(unknown_words_glove))

max_features = max_features or len(tokenizer.word_index) + 1

embedding_matrix = np.concatenate([crawl_matrix, glove_matrix], axis=-1)

del crawl_matrix
del glove_matrix
gc.collect()

y_train_torch = torch.tensor(np.hstack([y_train, y_aux_train]), dtype=torch.float32)

n unknown words (crawl):  335
n unknown words (glove):  355


# Sequence Bucketing

*   Use pad_sequences to convert the sequences into 2-D numpy array

In [11]:
x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)

In [12]:
lengths = torch.from_numpy(np.array([len(x) for x in x_train]))

#max_word_len = lengths.max() 
x_train_padded = torch.from_numpy(sequence.pad_sequences(x_train, maxlen=max_word_len))
x_train_padded.shape

torch.Size([1000, 300])

# Training

* To use sequence bucketing with maximum length.

The validation dataset is only added so that the fast.ai DataBunch works as expected and it consists of only 2 samples.

In [13]:
test_lengths = torch.from_numpy(np.array([len(x) for x in x_test]))
x_test_padded = torch.from_numpy(sequence.pad_sequences(x_test, maxlen=max_word_len))

In [14]:
validation_split = 0.2
shuffle_dataset = True
random_seed= 42

# Creating data indices for training and validation splits:
dataset_size = len(x_train_padded)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

In [15]:
def custom_loss(data, targets):
    ''' Define custom loss function for weighted BCE on 'target' column '''
    bce_loss_1 = nn.BCEWithLogitsLoss(weight=targets[:,1:2])(data[:,:1],targets[:,:1])
    bce_loss_2 = nn.BCEWithLogitsLoss()(data[:,1:],targets[:,2:])
    return (bce_loss_1 * loss_weight) + bce_loss_2

In [16]:
random_state = 42
skf = StratifiedKFold(n_splits = NUM_SPLITS, shuffle=True, random_state=random_state)

In [ ]:
# TODO: this is wrong!
def threshold_search(y_proba, val_ind):
    best_threshold = 0
    best_score = 0
    for threshold in [i * 0.01 for i in range(100)]:
        
        y_true = np.where(train['target'] >= threshold, 1, 0)[val_ind]
        y_aux_true = np.where(train[identity_columns] >= threshold, 1, 0)[val_ind]
        
        if(len(set(y_true)) == 1): # can't calcualte aucroc if only one class
            continue
        
        evaluator = JigsawEvaluator(y_true, y_aux_true,overall_model_weight=1/NUM_SPLITS)
        score = evaluator.get_final_metric(y_proba)
        
        if score > best_score:
            best_threshold = threshold
            best_score = score
    return (best_threshold, best_score)

In [18]:
make_weights_dir(MODEL_DIR)

oof_preds = np.zeros(train.shape[0])
feature_importance_df = pd.DataFrame()
final_auc = 0
thresholds = []
for fold, (trn_idxs, val_idxs) in enumerate(skf.split(x_train_padded,y_train_torch[:,0])):
    print("Current Fold: %s" % str(fold + 1))

    all_val_preds = []

    for model_idx in range(NUM_MODELS):
        full_train_dataset = data.TensorDataset(x_train_padded, lengths, y_train_torch)
        train_dataset = data.Subset(full_train_dataset, indices=trn_idxs)
        valid_dataset = data.Subset(full_train_dataset, indices=val_idxs)

        train_collator = SequenceBucketCollator(lambda lenghts: lenghts.max(),
                                                max_word_len,
                                                sequence_index=0, 
                                                length_index=1, 
                                                label_index=2)

        train_loader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=train_collator)
        valid_loader = data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, collate_fn=train_collator)

        databunch = DataBunch(train_dl=train_loader, valid_dl=valid_loader, collate_fn=train_collator)
        
        
        print('Model: %s' % model_idx)
        model_name = MODEL_DIR + ("f%s_m%s" % (fold,model_idx))
        seed_everything(1 + model_idx)
        model = NeuralNet(embedding_matrix, y_aux_train.shape[-1])
        learn = Learner(databunch, model, loss_func=custom_loss, model_dir=model_name + "w")
        val_preds = train_model(learn,valid_dataset, output_dim=7)    
        all_val_preds.append(val_preds)
        
        # saving the model
        model_sin_emb = model.state_dict()
        del model_sin_emb['embedding.weight']
        #learn.save('final_model')
        torch.save(model_sin_emb, model_name + "m")
        
        del learn
        del model
        del val_preds
        gc.collect()
        torch.cuda.empty_cache()

    y_preds = np.mean(all_val_preds, axis=0)[:, 0]
    kth_threshold, auc_score = threshold_search(y_preds,val_idxs)
    thresholds.append(kth_threshold)
    
    print('Final Kaggle Score: ', auc_score)
    final_auc += auc_score
print('Final ROC score: ', final_auc/NUM_SPLITS)

save_model_stats(MODEL_NOTES,
                 NUM_SPLITS,
                 MODEL_DIR,
                 NUM_MODELS,
                 notebook_start_time,
                 final_auc/NUM_SPLITS,
                 thresholds)

Current Fold: 1
Model: 0


epoch,train_loss,valid_loss,time
0,1.387475,1.191934,00:00


epoch,train_loss,valid_loss,time
0,1.180861,1.021706,00:00


epoch,train_loss,valid_loss,time
0,1.011504,0.861754,00:00


epoch,train_loss,valid_loss,time
0,0.852487,0.729816,00:00


...

epoch,train_loss,valid_loss,time
0,1.010318,0.839041,00:00


epoch,train_loss,valid_loss,time
0,0.847677,0.699813,00:00


epoch,train_loss,valid_loss,time
0,0.708602,0.644020,00:00


/home/curtis/Desktop/kaggle/jigsaw/notebooks/jig_utils.py:255: RuntimeWarning: divide by zero encountered in power
  total = sum(np.power(array, self.power))
/home/curtis/Desktop/kaggle/jigsaw/notebooks/jig_utils.py:255: RuntimeWarning: divide by zero encountered in power
  total = sum(np.power(array, self.power))
/home/curtis/Desktop/kaggle/jigsaw/notebooks/jig_utils.py:255: RuntimeWarning: divide by zero encountered in power
  total = sum(np.power(array, self.power))
/home/curtis/Desktop/kaggle/jigsaw/notebooks/jig_utils.py:255: RuntimeWarning: divide by zero encountered in power
  total = sum(np.power(array, self.power))
/home/curtis/Desktop/kaggle/jigsaw/notebooks/jig_utils.py:255: RuntimeWarning: divide by zero encountered in power
  total = sum(np.power(array, self.power))
/home/curtis/Desktop/kaggle/jigsaw/notebooks/jig_utils.py:255: RuntimeWarning: divide by zero encountered in power
  total = sum(np.power(array, self.power))
/home/curtis/Desktop/kaggle/jigsaw/notebooks/jig_uti

Final Kaggle Score:  0
Final ROC score:  0.0
saved model to ../models/v5_3_2_1/
{'model_notes': '', 'num_folds': 2, 'models_per_fold': 2, 'time_ran': '2019-06-06 23:57:03.745506', 'run_time': 30.637471199035645, 'final_val': 0.0, 'thresholds': [0, 0]}
